## Instalação de bibliotecas

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.4 MB/s eta 0:00:00


In [ ]:
import random
import torch
import torch.nn.functional as F
import numpy as np

Definição da semente para aleatoriedade, para fins de reprodução do treinamento.

In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

## Download e processamento das bases (treinamento e teste)

In [ ]:
!wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv

--2023-03-10 22:48:20--  https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.212.128, 173.194.216.128, 173.194.217.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8076179 (7.7M) [text/tab-separated-values]
Saving to: ‘msmarco_triples.train.tiny.tsv’

msmarco_triples.tra 100%[===================>]   7.70M  --.-KB/s    in 0.04s   

2023-03-10 22:48:20 (217 MB/s) - ‘msmarco_triples.train.tiny.tsv’ saved [8076179/8076179]



In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('msmarco_triples.train.tiny.tsv', sep='\t', header=None)

In [ ]:
df.head()

,0,1,2
0,is a little caffeine ok during pregnancy,We donât know a lot about the effects of caf...,It is generally safe for pregnant women to eat...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ..."
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ..."


In [ ]:
df_pos = pd.DataFrame()
df_neg = pd.DataFrame()

for index, row in df.iterrows():
  df_pos = df_pos.append({"query":row[0], "passage":row[1], "score":1}, ignore_index=True)
  df_neg = df_neg.append({"query":row[0], "passage":row[2], "score":0}, ignore_index=True)
    

In [ ]:
df_pos

,query,passage,score
0,is a little caffeine ok during pregnancy,We donât know a lot about the effects of caf...,1.0
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,1.0
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,1.0
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,1.0
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...",1.0
...,...,...,...
10995,where is maramal in skyrim,I'm having a rather difficult time in Skyrim t...,1.0
10996,which is a basic unit of a sugar molecule,The basic unit of carbohydrate is a single sug...,1.0
10997,deloitte consulting or phone number,A: The phone number for Deloitte Consulting Ll...,1.0
10998,"how many hours a day to work to get lunch in ,a","In the United States, most states require a mi...",1.0


In [ ]:
df_neg

,query,passage,score
0,is a little caffeine ok during pregnancy,It is generally safe for pregnant women to eat...,0.0
1,what fruit is native to australia,"The kola nut is the fruit of the kola tree, a ...",0.0
2,how large is the canadian military,The Canadian Physician Health Institute (CPHI)...,0.0
3,types of fruit trees,"The kola nut is the fruit of the kola tree, a ...",0.0
4,how many calories a day are lost breastfeeding,"However, you still need some niacin each day; ...",0.0
...,...,...,...
10995,where is maramal in skyrim,One type of character isnât necessarily bett...,0.0
10996,which is a basic unit of a sugar molecule,According to Merriam-Webster and the Online Et...,0.0
10997,deloitte consulting or phone number,Virtual phone numbers are convenient for both ...,0.0
10998,"how many hours a day to work to get lunch in ,a",1 Levaquin can be taken at any time of the day...,0.0


In [ ]:
from sklearn.model_selection import train_test_split

X_train_pos = df_pos.drop("score", axis=1)
Y_train_pos = df_pos["score"]

X_train_pos, X_val_pos, Y_train_pos, Y_val_pos = train_test_split(X_train_pos, Y_train_pos, test_size=0.2, random_state=42)

X_train_neg = df_neg.drop("score", axis=1)
Y_train_neg = df_neg["score"]

X_train_neg, X_val_neg, Y_train_neg, Y_val_neg = train_test_split(X_train_neg, Y_train_neg, test_size=0.2, random_state=42)

In [ ]:
X_train_pos

,query,passage
10735,"how far is cumberland md from washington, dc","Driving distance from Washington, DC to Cumber..."
5937,average cost to move and set up mobile home,"1 On average, a short mobile home move that is..."
7642,how big can siberian huskies get,male husky can get 23 feet long and females ca...
3328,how long can you hold someone's attention,"So much so, that as you approach the pulpit, t..."
8681,hcpcs code for medication management,"Inpatient telehealth pharmacologic management,..."
...,...,...
5734,how much did vanderbilt breakers mansion cost ...,"The mansion cost $11 million ($260,000,000 in ..."
5191,weather in dunedin,Dunedin Average Weather. 1 The highest monthl...
5390,meaning of the name brid,The name Brid is a Celtic baby name. In Celtic...
860,meaning of trap femboy,"From what I can see, trap as a noun meaning sp..."


In [ ]:
Y_train_pos

10735    1.0
5937     1.0
7642     1.0
3328     1.0
8681     1.0
        ... 
5734     1.0
5191     1.0
5390     1.0
860      1.0
7270     1.0
Name: score, Length: 8800, dtype: float64

In [ ]:
X_val_pos

,query,passage
107,where to get corso riggs,Background. Always cheerful and eternally opti...
5484,what rivers flow into the wabash river,The Wabash River rises south of Grand Lake abo...
6998,where is fiji islands,1 The Fiji Islands are an archipelago of over ...
3984,how long do nipple piercings take to heal,"Typically, nipple piercings take anywhere from..."
3111,what is a pole pocket banner,Pole pockets on banners are something that can...
...,...,...
5284,what are bacterial organisms,Taxonomy. Bacteria are members of the prokaryo...
3909,what county is park ridge nj,"Park Ridge, New Jersey. Park Ridge is a boroug..."
10549,what kind of tea gets rid of water retention,"To get rid of water retention, you must follow..."
5809,bordeaux meaning,Bordeaux is a port city on the Garonne River i...


In [ ]:
Y_val_pos

107      1.0
5484     1.0
6998     1.0
3984     1.0
3111     1.0
        ... 
5284     1.0
3909     1.0
10549    1.0
5809     1.0
7571     1.0
Name: score, Length: 2200, dtype: float64

In [ ]:
X_train_neg

,query,passage
10735,"how far is cumberland md from washington, dc",Averages for Washington DC in September. The w...
5937,average cost to move and set up mobile home,The average cost per square foot to build a ho...
7642,how big can siberian huskies get,If a normal tour of duty to Siberia is 12 mont...
3328,how long can you hold someone's attention,Can't take my eyes off of you. You'd be like h...
8681,hcpcs code for medication management,"In addition, Samsung created a Supplier Code o..."
...,...,...
5734,how much did vanderbilt breakers mansion cost ...,The average cost per square foot to build a ho...
5191,weather in dunedin,"Weather for Bradford, IL - Bradford, IL 61421 ..."
5390,meaning of the name brid,German Meaning: The name Jay is a German baby ...
860,meaning of trap femboy,Full Definition of DEATH TRAP. : a structure o...


In [ ]:
Y_train_neg

10735    0.0
5937     0.0
7642     0.0
3328     0.0
8681     0.0
        ... 
5734     0.0
5191     0.0
5390     0.0
860      0.0
7270     0.0
Name: score, Length: 8800, dtype: float64

In [ ]:
X_val_neg

,query,passage
107,where to get corso riggs,show more I have a male Cane Corso. I have rea...
5484,what rivers flow into the wabash river,"As such, the city's culture is also closely ti..."
6998,where is fiji islands,"On 24 July 2003, six member states of the Paci..."
3984,how long do nipple piercings take to heal,Fluid that leaks from one or both nipples is c...
3111,what is a pole pocket banner,The Geographic North Pole is located approxima...
...,...,...
5284,what are bacterial organisms,Cucumber bacterial wilt is transmitted by the ...
3909,what county is park ridge nj,It was a sleazy thing to do. I complained to A...
10549,what kind of tea gets rid of water retention,Hereâs the way you make a great cup of tulsi...
5809,bordeaux meaning,The capital of France is Paris. Largest Cities...


In [ ]:
Y_val_neg

107      0.0
5484     0.0
6998     0.0
3984     0.0
3111     0.0
        ... 
5284     0.0
3909     0.0
10549    0.0
5809     0.0
7571     0.0
Name: score, Length: 2200, dtype: float64

In [ ]:
X_train = pd.concat([X_train_pos, X_train_neg], axis=0, ignore_index=True)
Y_train = pd.concat([Y_train_pos, Y_train_neg], axis=0, ignore_index=True)
X_val = pd.concat([X_val_pos, X_val_neg], axis=0, ignore_index=True)
Y_val = pd.concat([Y_val_pos, Y_val_neg], axis=0, ignore_index=True)

In [ ]:
X_train

,query,passage
0,"how far is cumberland md from washington, dc","Driving distance from Washington, DC to Cumber..."
1,average cost to move and set up mobile home,"1 On average, a short mobile home move that is..."
2,how big can siberian huskies get,male husky can get 23 feet long and females ca...
3,how long can you hold someone's attention,"So much so, that as you approach the pulpit, t..."
4,hcpcs code for medication management,"Inpatient telehealth pharmacologic management,..."
...,...,...
17595,how much did vanderbilt breakers mansion cost ...,The average cost per square foot to build a ho...
17596,weather in dunedin,"Weather for Bradford, IL - Bradford, IL 61421 ..."
17597,meaning of the name brid,German Meaning: The name Jay is a German baby ...
17598,meaning of trap femboy,Full Definition of DEATH TRAP. : a structure o...


In [ ]:
Y_train

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
17595    0.0
17596    0.0
17597    0.0
17598    0.0
17599    0.0
Name: score, Length: 17600, dtype: float64

In [ ]:
X_val

,query,passage
0,where to get corso riggs,Background. Always cheerful and eternally opti...
1,what rivers flow into the wabash river,The Wabash River rises south of Grand Lake abo...
2,where is fiji islands,1 The Fiji Islands are an archipelago of over ...
3,how long do nipple piercings take to heal,"Typically, nipple piercings take anywhere from..."
4,what is a pole pocket banner,Pole pockets on banners are something that can...
...,...,...
4395,what are bacterial organisms,Cucumber bacterial wilt is transmitted by the ...
4396,what county is park ridge nj,It was a sleazy thing to do. I complained to A...
4397,what kind of tea gets rid of water retention,Hereâs the way you make a great cup of tulsi...
4398,bordeaux meaning,The capital of France is Paris. Largest Cities...


In [ ]:
Y_val

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
4395    0.0
4396    0.0
4397    0.0
4398    0.0
4399    0.0
Name: score, Length: 4400, dtype: float64

In [ ]:
X_train

,query,passage
0,"how far is cumberland md from washington, dc","Driving distance from Washington, DC to Cumber..."
1,average cost to move and set up mobile home,"1 On average, a short mobile home move that is..."
2,how big can siberian huskies get,male husky can get 23 feet long and females ca...
3,how long can you hold someone's attention,"So much so, that as you approach the pulpit, t..."
4,hcpcs code for medication management,"Inpatient telehealth pharmacologic management,..."
...,...,...
17595,how much did vanderbilt breakers mansion cost ...,The average cost per square foot to build a ho...
17596,weather in dunedin,"Weather for Bradford, IL - Bradford, IL 61421 ..."
17597,meaning of the name brid,German Meaning: The name Jay is a German baby ...
17598,meaning of trap femboy,Full Definition of DEATH TRAP. : a structure o...


In [ ]:
Y_train

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
17595    0.0
17596    0.0
17597    0.0
17598    0.0
17599    0.0
Name: score, Length: 17600, dtype: float64

In [ ]:
X_val

,query,passage
0,where to get corso riggs,Background. Always cheerful and eternally opti...
1,what rivers flow into the wabash river,The Wabash River rises south of Grand Lake abo...
2,where is fiji islands,1 The Fiji Islands are an archipelago of over ...
3,how long do nipple piercings take to heal,"Typically, nipple piercings take anywhere from..."
4,what is a pole pocket banner,Pole pockets on banners are something that can...
...,...,...
4395,what are bacterial organisms,Cucumber bacterial wilt is transmitted by the ...
4396,what county is park ridge nj,It was a sleazy thing to do. I complained to A...
4397,what kind of tea gets rid of water retention,Hereâs the way you make a great cup of tulsi...
4398,bordeaux meaning,The capital of France is Paris. Largest Cities...


In [ ]:
Y_val

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
4395    0.0
4396    0.0
4397    0.0
4398    0.0
4399    0.0
Name: score, Length: 4400, dtype: float64

## Finetuning

Preparação dos dados, com tokenização e construção do dataset e dataloaders.

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'microsoft/MiniLM-L12-H384-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from statistics import mean, stdev

lengths = [len(tokens) for tokens in tokenizer(list(X_train["passage"])[:1_000])['input_ids']]
print(f'Mean length in tokens: {mean(lengths):0.2f}')
print(f'Stdev length in tokens: {stdev(lengths):0.2f}')

Mean length in tokens: 80.45
Stdev length in tokens: 31.20


In [ ]:
max_length = 512

In [ ]:
train_queries = list(X_train["query"])
train_passages = list(X_train["passage"])
val_queries = list(X_val["query"])
val_passages = list(X_val["passage"])

train_queries_tokenized = tokenizer(train_queries, max_length=max_length, truncation=True)
train_passages_tokenized = tokenizer(train_passages, max_length=max_length, truncation=True)
val_queries_tokenized = tokenizer(val_queries, max_length=max_length, truncation=True)
val_passages_tokenized = tokenizer(val_passages, max_length=max_length, truncation=True)

In [ ]:
train_queries_tokenized[0]

Encoding(num_tokens=11, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
train_queries_tokenized[0].ids

[101, 2129, 2521, 2003, 12310, 9108, 2013, 2899, 1010, 5887, 102]

In [ ]:
train_queries_tokenized[0].tokens

['[CLS]',
 'how',
 'far',
 'is',
 'cumberland',
 'md',
 'from',
 'washington',
 ',',
 'dc',
 '[SEP]']

In [ ]:
len(train_queries_tokenized[0].tokens)

11

In [ ]:
len(train_queries_tokenized['input_ids'][0])

11

In [ ]:
train_queries_tokenized[0].attention_mask

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
len(train_queries_tokenized[0].attention_mask)

11

In [ ]:
from torch.utils import data

class Dataset(data.Dataset):
    def __init__(self, queries, passages, targets):
        self.queries = queries
        self.passages = passages
        self.targets = targets
    
    def __len__(self):
        return len(self.queries['input_ids'])
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.queries['input_ids'][idx] + self.passages['input_ids'][idx],
            'attention_mask': self.queries['attention_mask'][idx] + self.passages['attention_mask'][idx],
            'labels': int(self.targets[idx]),
        }



In [ ]:
dataset_train = Dataset(train_queries_tokenized, train_passages_tokenized, Y_train)
assert len(dataset_train[0]['input_ids']) > 0
assert len(dataset_train[1]['attention_mask']) > 0
assert type(dataset_train[2]['labels']) == int

In [ ]:
dataset_valid = Dataset(val_queries_tokenized, val_passages_tokenized, Y_val)

In [ ]:
from transformers import BatchEncoding

# Tokens do tipo "pad" para textos com tamanho inferior ao máximo suportado.
def collate_fn(batch):
  return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))



In [ ]:
dataloader_train = data.DataLoader(dataset_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
dataloader_valid = data.DataLoader(dataset_valid, batch_size=32, shuffle=False, collate_fn=collate_fn)

for batch in dataloader_train:
    assert batch['input_ids'].shape[0] <= dataloader_train.batch_size
    assert batch['input_ids'].shape[1] <= max_length
    break

### Training Loop

In [ ]:
def evaluate(model, dataloader, set_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            outputs = model(**batch.to(device))
            loss_val = outputs.loss
            losses.append(loss_val.cpu().item())
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == batch['labels']).sum().item()

    print(f'{set_name} loss: {mean(losses):0.3f}; {set_name} accuracy: {correct / len(dataloader.dataset):0.3f}')

In [ ]:
from torch import nn
from torch import optim
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
print('Parameters', model.num_parameters())

epochs = 5
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
num_training_steps = epochs * len(dataloader_train)

num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

# Training loop
for epoch in tqdm(range(epochs), desc='Epochs'):
    model.train()
    train_losses = []
    for batch in tqdm(dataloader_train, mininterval=0.5, desc='Train', disable=False):
        optimizer.zero_grad()
        outputs = model(**batch.to(device))
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_losses.append(loss.cpu().item())

    print(f'Epoch: {epoch + 1} Training loss: {mean(train_losses):0.2f}')
    evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.694; Valid accuracy: 0.500


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

...

Valid loss: 0.224; Valid accuracy: 0.934


Train:   0%|          | 0/550 [00:00<?, ?it/s]

Epoch: 5 Training loss: 0.04


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.247; Valid accuracy: 0.937


In [ ]:
MODELS_PATH = '/content/gdrive/MyDrive/Unicamp-aula-3'


In [ ]:
model.save_pretrained(f'{MODELS_PATH}/models_ranker')
tokenizer.save_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

## Reranqueamento

In [ ]:
main_path = '/content/gdrive/MyDrive/Unicamp-aula-2/'

Baixando o repositório do Pyserini para ter acesso ao dataset do TREC DL 20 e à ferramenta de avaliação.

In [146]:
!git clone https://github.com/castorini/pyserini.git --recurse-submodules

Cloning into 'pyserini'...
remote: Enumerating objects: 7262, done.
remote: Counting objects: 100% (308/308), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 7262 (delta 184), reused 206 (delta 124), pack-reused 6954
Receiving objects: 100% (7262/7262), 4.32 MiB | 3.24 MiB/s, done.
Resolving deltas: 100% (5230/5230), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/pyserini/tools'...
remote: Enumerating objects: 723, done.        
remote: Counting objects: 100% (480/480), done.        
remote: Compressing objects: 100% (418/418), done.        
remote: Total 723 (delta 74), reused 453 (delta 61), pack-reused 243        
Receiving objects: 100% (723/723), 57.78 MiB | 4.53 MiB/s, done.
Resolving deltas: 100% (158/158), done.
Submodule path 'tools': checked out '7b84f773225b5973b4533dfa0aa18653409a6146'


In [147]:
!cd pyserini/tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!cd pyserini/tools/eval/ndeval && make && cd ../../..

trec_eval.9.0.4/
trec_eval.9.0.4/m_prefs_pair.c
trec_eval.9.0.4/m_ndcg_p.c
trec_eval.9.0.4/m_infap.c
trec_eval.9.0.4/m_num_q.c
trec_eval.9.0.4/m_iprec_at_recall.c
trec_eval.9.0.4/form_prefs_counts.c
trec_eval.9.0.4/m_prefs_num_prefs_ful_ret.c
trec_eval.9.0.4/utility_pool.c
trec_eval.9.0.4/m_binG.c
trec_eval.9.0.4/meas_avg.c
trec_eval.9.0.4/m_gm_bpref.c
trec_eval.9.0.4/m_runid.c
trec_eval.9.0.4/m_bpref.c
trec_eval.9.0.4/m_gm_map.c
trec_eval.9.0.4/trec_eval.h
trec_eval.9.0.4/m_yaap.c
trec_eval.9.0.4/m_relstring.c
trec_eval.9.0.4/m_Rprec.c
trec_eval.9.0.4/m_prefs_avgjg.c
trec_eval.9.0.4/m_success.c
trec_eval.9.0.4/m_ndcg.c
trec_eval.9.0.4/functions.h
trec_eval.9.0.4/m_P_avgjg.c
trec_eval.9.0.4/test/
trec_eval.9.0.4/test/qrels.rel_level
trec_eval.9.0.4/test/results.test
trec_eval.9.0.4/test/qrels.test
trec_eval.9.0.4/test/out.test.qrels_jg
trec_eval.9.0.4/test/out.test.meas_params
trec_eval.9.0.4/test/out.test.a
trec_eval.9.0.4/test/out.test.prefs
trec_eval.9.0.4/test/out.test.aqcM
trec_ev

In [148]:
!mv pyserini {root_dir}

### Indexação

In [ ]:
!python pyserini/tools/scripts/msmarco/convert_collection_to_jsonl.py \
 --collection-path {main_path}/collections/msmarco-passage/collection.tsv \
 --output-folder collections/msmarco-passage/collection_jsonl

Converting collection...
Converted 0 docs, writing into file 1
Converted 100,000 docs, writing into file 1
Converted 200,000 docs, writing into file 1
Converted 300,000 docs, writing into file 1
Converted 400,000 docs, writing into file 1
Converted 500,000 docs, writing into file 1
Converted 600,000 docs, writing into file 1
Converted 700,000 docs, writing into file 1
Converted 800,000 docs, writing into file 1
Converted 900,000 docs, writing into file 1
Converted 1,000,000 docs, writing into file 2
Converted 1,100,000 docs, writing into file 2
Converted 1,200,000 docs, writing into file 2
Converted 1,300,000 docs, writing into file 2
Converted 1,400,000 docs, writing into file 2
Converted 1,500,000 docs, writing into file 2
Converted 1,600,000 docs, writing into file 2
Converted 1,700,000 docs, writing into file 2
Converted 1,800,000 docs, writing into file 2
Converted 1,900,000 docs, writing into file 2
Converted 2,000,000 docs, writing into file 3
Converted 2,100,000 docs, writing i

In [ ]:
!python -m spacy download en_core_web_sm

2023-03-11 16:26:13.671089: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 16:26:14.598529: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-11 16:26:14.598638: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-11 16:26:14.598666: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [ ]:
!git clone https://github.com/castorini/anserini.git --recurse-submodules

Cloning into 'anserini'...
remote: Enumerating objects: 25677, done.
remote: Counting objects: 100% (1273/1273), done.
remote: Compressing objects: 100% (696/696), done.
remote: Total 25677 (delta 784), reused 907 (delta 510), pack-reused 24404
Receiving objects: 100% (25677/25677), 79.69 MiB | 22.63 MiB/s, done.
Resolving deltas: 100% (16516/16516), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/anserini/tools'...
remote: Enumerating objects: 723, done.        
remote: Counting objects: 100% (480/480), done.        
remote: Compressing objects: 100% (418/418), done.        
remote: Total 723 (delta 74), reused 453 (delta 61), pack-reused 243        
Receiving objects: 100% (723/723), 57.78 MiB | 23.67 MiB/s, done.
Resolving deltas: 100% (158/158), done.
Submodule path 'tools': checked out '95fbaf2af75e2b59304ac5702d5479d50f3bd9ef'


In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


In [ ]:
!conda install -c conda-forge maven

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [ ]:
!mvn -f anserini/pom.xml clean package appassembler:assemble

[INFO] Scanning for projects...
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.8.1/maven-compiler-plugin-3.8.1.pom (12 kB at 35 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/33/maven-plugins-33.pom (11 kB at 534 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/33/maven-parent-33.pom (44 kB at 1.1 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/21/apache-21.pom (17 kB at 1.6 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.8.1/maven-compiler-plugin-3.8.1.jar (62 kB at 1.7 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/mojo/appassembler-maven-plugin/2.1.0/appassembler-maven-plugin-2.1.0.pom (19 kB at 1.2 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/code

In [ ]:
!cp anserini/target/anserini-0.20.1-SNAPSHOT-fatjar.jar pyserini/pyserini/resources/jars

In [ ]:
!pip install -e pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/pyserini
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 123.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 119.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 122.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input collections/msmarco-passage/collection_jsonl \
  --index indexes/lucene-index-msmarco-passage \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-03-11 16:38:46,786 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-03-11 16:38:46,789 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-03-11 16:38:46,789 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-03-11 16:38:46,789 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: collections/msmarco-passage/collection_jsonl
2023-03-11 16:38:46,789 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-03-11 16:38:46,790 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-03-11 16:38:46,790 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-03-11 16:38:46,790 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en
2023-03-11 16:38:46,790 INFO  [main] 

### Busca

In [167]:
topics = 'dl20'
qrels_eval = 'dl20-passage'
qrels_eval_file = 'pyserini/tools/topics-and-qrels/qrels.dl20-passage.txt'
topics_file = 'pyserini/tools/topics-and-qrels/topics.dl20.txt'

In [ ]:
!python3 pyserini/tools/scripts/msmarco/filter_queries.py \
--qrels <(sed -e 's/ /\t/g' {qrels_eval_file}) \
--queries {topics_file} \
--output topics.dl20.small.tsv

Done!


In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!conda install faiss-cpu -c pytorch

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - done

# All requested packages already installed.



In [ ]:
!python -m pyserini.search.lucene \
  --index indexes/lucene-index-msmarco-passage \
  --topics topics.dl20.small.tsv \
  --output runs/run.{topics}.bm25tuned.txt \
  --output-format msmarco \
  --hits 1000 \
  --bm25 --k1 0.82 --b 0.68

Setting BM25 parameters: k1=0.82, b=0.68
Running topics.dl20.small.tsv topics, saving to runs/run.dl20.bm25tuned.txt...
100% 54/54 [00:08<00:00,  6.69it/s]


In [ ]:
!python -m pyserini.eval.convert_msmarco_run_to_trec_run \
   --input runs/run.{topics}.bm25tuned.txt \
   --output runs/run.{topics}.bm25tuned.trec

Done!


In [ ]:
!python pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_qrels.py \
   --input {qrels_eval_file} \
   --output collections/msmarco-passage/qrels.{qrels_eval}.trec

Done!


In [ ]:
!pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   collections/msmarco-passage/qrels.{qrels_eval}.trec runs/run.{topics}.bm25tuned.trec

map                   	all	0.2876
ndcg_cut_10           	all	0.4876


### Etapa de reranqueamento (estágio 2)

In [ ]:
root_dir = '/content/gdrive/MyDrive/Unicamp-aula-3/'

In [ ]:
#Carrega os ids e textos das queries em memória
df_queries = pd.read_csv(f'{root_dir}topics.dl20.small.tsv', sep='\t', header=None)

In [ ]:
df_queries.head()

,0,1
0,1030303,who is aziz hashim
1,1037496,who is rep scalise?
2,1043135,who killed nicholas ii of russia
3,1051399,who sings monk theme song
4,1064670,why do hunters pattern their shotguns?


In [ ]:
collection_path = '/content/gdrive/MyDrive/Unicamp-aula-2/collections/msmarco-passage/collection.tsv'
df_passages = pd.read_csv(collection_path, sep='\t', header=None)

In [ ]:
#Dúvida: não sei se é realmente necessário carregar o mesmo tokenizador usado no 
#finetuning, porque creio que ele não foi modificado.
tokenizer = AutoTokenizer.from_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

In [104]:
query_ids = []
queries = []
passage_ids = []
passages = []

with open(f'{root_dir}runs/run.dl20.bm25tuned.txt') as f:
  for line in f:
      fields = line.strip().split('\t')
      query_id = fields[0]
      query_ids.append(query_id)
      query_text = df_queries[df_queries.iloc[:,0] == int(query_id)].iloc[0,1]
      queries.append(query_text)
      passage_id = fields[1]
      passage_ids.append(passage_id)
      passage_text = df_passages[df_passages.iloc[:,0] == int(passage_id)].iloc[0,1]
      passages.append(passage_text)

tokenized_queries = tokenizer(queries, max_length=max_length, truncation=True)
tokenized_passages = tokenizer(passages, max_length=max_length, truncation=True)
      

In [105]:
# Aqui, pouco importa o target, porque ele não será utilizado para calcular acurácia.  Por isso coloquei todos iguais 
# a 1 (relevantes)
dataset_test = Dataset(tokenized_queries, tokenized_passages, [1]*len(tokenized_queries['input_ids']))

In [106]:
dataloader_test = data.DataLoader(dataset_test, batch_size=32, shuffle=False, collate_fn=collate_fn)


In [107]:
def evaluate_test_dataset(model, dataloader, set_name):
    scores = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            outputs = model(**batch.to(device))
            pos_score = torch.softmax(outputs.logits,1)[:,1]
            scores = scores + pos_score.tolist()
    return scores

In [108]:
model = AutoModelForSequenceClassification.from_pretrained(f'{MODELS_PATH}/models_ranker').to(device)

In [109]:
scores = evaluate_test_dataset(model=model, dataloader=dataloader_test, set_name='Test')

Test:   0%|          | 0/1688 [00:00<?, ?it/s]

In [110]:
len(scores)

54000

In [126]:
len(passage_ids)

54000

In [128]:
zipped_results = []

#Por alguma razão misteriosa, o zip do Python não funcionou, deixou a lista 
#vazia ou impossível de ser iterada.
for i, query_id in enumerate(query_ids):
  zipped_results.append((query_id, passage_ids[i], scores[i]))

In [136]:
#Quebra a lista em sublistas por query
prev_query_id = -1
sublists = []
current_list = []

for query_id, passage_id, score in zipped_results:
  if query_id != prev_query_id:
    if len(current_list) > 0:
      sublists.append(current_list)
      current_list = []
  current_list.append((query_id, passage_id, score))
  prev_query_id = query_id

if len(current_list) > 0:
  sublists.append(current_list)


In [137]:
len(sublists)

54

In [139]:
len(sublists[0])

1000

Ordena cada sublista

In [140]:
sorted_list = []

for sublist in sublists:
  sorted_sublist = sorted(sublist, key=lambda x: x[2], reverse=True)
  sorted_list += sorted_sublist

In [141]:
len(sorted_list)

54000

In [142]:
with open("run.dl20.bert_reranked.trec", "w") as f:
  for i, (query_id, passage_id, score) in enumerate(sorted_list):
    f.write(f'{query_id}\tQ0\t{passage_id}\t{i+1}\t{score}\tbert_reranked\n')

In [143]:
!head run.dl20.bert_reranked.trec

23849	Q0	8010558	1	0.99784255027771	bert_reranked
23849	Q0	2647769	2	0.9977450370788574	bert_reranked
23849	Q0	8010559	3	0.9977275729179382	bert_reranked
23849	Q0	8010561	4	0.996823787689209	bert_reranked
23849	Q0	6947924	5	0.17504705488681793	bert_reranked
23849	Q0	4091551	6	0.10943090170621872	bert_reranked
23849	Q0	2017213	7	0.03757953271269798	bert_reranked
23849	Q0	8246990	8	0.028255965560674667	bert_reranked
23849	Q0	4834498	9	0.02185242436826229	bert_reranked
23849	Q0	1215557	10	0.011535624042153358	bert_reranked


In [151]:
root_dir

'/content/gdrive/MyDrive/Unicamp-aula-3/'

In [168]:
qrels_eval

'dl20-passage'

In [162]:
!chmod 755 {root_dir}/pyserini/tools/eval/trec_eval.9.0.4/trec_eval

In [170]:
!/content/gdrive/MyDrive/Unicamp-aula-3/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   {root_dir}collections/msmarco-passage/qrels.{qrels_eval}.trec run.dl20.bert_reranked.trec

map                   	all	0.4340
ndcg_cut_10           	all	0.6210


Salvamento de arquivos baixados e gerados

In [171]:
#!mv anserini {root_dir}
#!mv collections {root_dir}
#!mv indexes {root_dir}
#!mv runs {root_dir}
#!mv topics.dl20.small.tsv {root_dir}
#!mv topics.dl20.small.txt {root_dir}
!cp run.dl20.bert_reranked.trec {root_dir}